In [1]:
import os
import sys 
import pandas as pd
import yaml
import numpy as np
import time
import pickle
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
from torch import nn, Tensor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
import lightning as pl
from lightning.pytorch.loggers import CSVLogger
import torchvision.models as models

# this needs to be installed from https://github.com/jacobgil/confidenceinterval
import confidenceinterval as ci

IM_SIZE = 224
EPOCHS = 1000
NUM_WORKERS = 60 

torch.set_num_threads(NUM_WORKERS*2) 
torch.manual_seed(1120)
torch.set_float32_matmul_precision("medium")

import sys
sys.path.append("../")

from modules import define_df, retrieve_cancer_status

/mnt/srl-oahu-1/srl-hawaii-1/ariannab/anaconda3/envs/torch_optuna/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# loading in the predictions
test_input_file = "sample_data/predictions.pkl"
with open(test_input_file, 'rb') as f:
    test_predictions = pickle.load(f)

test_densenet = define_df(['densenet_preds'], test_predictions)
test_densenet_pat = define_df(['densenet_preds'], test_predictions, patient_level=True)
test_densenet_pat['prediction_single'] = np.argmax(test_densenet_pat[['A_densenet_preds', 'B_densenet_preds', 'C_densenet_preds', 'D_densenet_preds']].to_numpy(), axis=1)

# original radiologist assignments 
testing_raw = pd.read_csv("sample_data/sample_data.csv")

test_densenet_pat['cancer'] = test_densenet_pat['ANALYSIS_ID'].apply(lambda x: retrieve_cancer_status(x, testing_raw))
test_densenet['cancer'] = test_densenet['ANALYSIS_ID'].apply(lambda x: retrieve_cancer_status(x, testing_raw))

In [16]:
new_samples = {'ANALYSIS_ID' : [], 'sample' : [],  'clinical_density' : [], 'BUS_density' : [], 'cancer' : [], 'age' : []}

for x in test_densenet_pat['ANALYSIS_ID'].tolist():
    new_samples['ANALYSIS_ID'].extend([x] * 100)
    new_samples['sample'].extend(list(range(100)))
    new_samples['cancer'].extend(test_densenet_pat[test_densenet_pat['ANALYSIS_ID'] == x]['cancer'].tolist() * 100)
    new_samples['age'].extend(test_densenet_pat[test_densenet_pat['ANALYSIS_ID'] == x]['age_us'].tolist() * 100)
    new_samples['clinical_density'].extend((test_densenet_pat[test_densenet_pat['ANALYSIS_ID'] == x]['labels'] - 1).tolist()* 100)

    bus_probs = np.round(test_densenet_pat[test_densenet_pat['ANALYSIS_ID'] == x][['A_densenet_preds', 'B_densenet_preds', 'C_densenet_preds', 'D_densenet_preds']].to_numpy()[0], 3)
    
    if(sum(bus_probs) != 1):
        bus_probs[1] = bus_probs[1] + ( 1 - sum(bus_probs))

    new_samples['BUS_density'].extend(np.random.choice(4, size=100, p=bus_probs))

In [18]:
# creating our new sampled dataframe, then deduplicating for construction of odds ratios
sampled_df = pd.DataFrame.from_dict(new_samples)
sampled_df_dedup = sampled_df.sample(frac=1.0).drop_duplicates(subset='ANALYSIS_ID', keep='first')

#### Predicting Cancer from predicted BUS BI-RADS density 
Because we're using the small sample dataset, we're not doing CV here. We did it with our full dataset in the paper. This is purely for illustrative purposes. Additionally, in the paper we construct odds ratios from `one_hot_encoded_df_dedup`. Due to the small size of the sample dataset, we are contructing from `one_hot_encoded_df`. **Both things must be amended if you want valid results**

In [24]:
one_hot_encoded_df = pd.get_dummies(sampled_df, columns=['BUS_density'], prefix='BUS_density')
# B is our reference category
X = one_hot_encoded_df[['age', 'BUS_density_0', 'BUS_density_2', 'BUS_density_3']].values
y = one_hot_encoded_df['cancer']

one_hot_encoded_df_dedup = pd.get_dummies(sampled_df_dedup, columns=['BUS_density'], prefix='BUS_density')

outer_cv = KFold(n_splits=3, shuffle=True, random_state=1120)

ct = ColumnTransformer([ ("passthrough", "passthrough", [1, 2, 3]), ('scaler', StandardScaler(), [0])])
pipe = Pipeline([('scaler', ct), ('model', LogisticRegression(random_state=1120, max_iter=10000, penalty=None, fit_intercept=True))])
pipe.fit(X=X, y=y)
nested_score = cross_val_score(pipe, X=X, y=y, cv=outer_cv, scoring='roc_auc_ovr')
preds = cross_val_predict(pipe, X=X, y=y, cv=outer_cv, method='predict_proba')

print(pipe['model'].coef_)
print(pipe['model'].intercept_)
print(pipe['model'].classes_)
print(nested_score.mean())

[[ 1.28568169 -1.66198019 -1.61066036 29.24126416]]
[-25.43176429]
[0 1]
1.0


In [26]:
preds_dedup = pipe.predict_proba(X=one_hot_encoded_df[['age', 'BUS_density_0', 'BUS_density_2', 'BUS_density_3']])
betas = np.insert(pipe['model'].coef_[0], 0, pipe['model'].intercept_).reshape((5, 1))
X = np.hstack([np.ones((400, 1)), ct.fit_transform(one_hot_encoded_df[['age', 'BUS_density_0', 'BUS_density_2', 'BUS_density_3']])])
W = np.diagflat(np.exp(np.dot(X, betas)) / ((1 + np.exp(np.dot(X, betas)))**2))
# define standard errors 
SE = np.sqrt(np.linalg.inv(np.dot(np.dot(np.transpose(X), W), X)))

/tmp/ipykernel_1159025/3758522736.py:8: RuntimeWarning: invalid value encountered in sqrt
  SE = np.sqrt(np.linalg.inv(np.dot(np.dot(np.transpose(X), W), X)))


In [27]:
# odds ratios for one SD increase in age 
# overflow is expected in the sample dataset
age_coef = pipe['model'].coef_[0][0]
print(np.exp(age_coef).round(2))
print(np.exp(age_coef - 1.96*(SE[1, 1])).round(2))
print(np.exp(age_coef + 1.96*(SE[1, 1])).round(2))

3.62
0.0
inf


/tmp/ipykernel_1159025/4157720483.py:5: RuntimeWarning: overflow encountered in exp
  print(np.exp(age_coef + 1.96*(SE[1, 1])).round(2))


In [28]:
# overflow is expected with sample dataset 
A_coef = pipe['model'].coef_[0][1]
print(np.exp(A_coef).round(2))
print(np.exp(A_coef - 1.96*(SE[2, 2])).round(2))
print(np.exp(A_coef + 1.96*(SE[2, 2])).round(2))
print("\n")

# overflow is expected with sample dataset 
C_coef = pipe['model'].coef_[0][2]
print(np.exp(C_coef).round(2))
print(np.exp(C_coef - 1.96*(SE[3, 3])).round(2))
print(np.exp(C_coef + 1.96*(SE[3, 3])).round(2))
print("\n")

# overflow is expected with sample dataset 
D_coef = pipe['model'].coef_[0][3]
print(np.exp(D_coef).round(2))
print(np.exp(D_coef - 1.96*(SE[4, 4])).round(2))
print(np.exp(D_coef + 1.96*(SE[4, 4])).round(2))

0.19
0.0
2.240449274008741e+144


0.2
0.0
inf


5004027270786.82
0.0
4.220613872876127e+163


/tmp/ipykernel_1159025/3218370448.py:10: RuntimeWarning: overflow encountered in exp
  print(np.exp(C_coef + 1.96*(SE[3, 3])).round(2))
